Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
# logistic models
batch_size = 128
lamb = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
    + lamb * tf.nn.l2_loss(weights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 22.227779
Minibatch accuracy: 10.2%
Validation accuracy: 16.5%
Minibatch loss at step 500: 2.589151
Minibatch accuracy: 78.1%
Validation accuracy: 76.1%
Minibatch loss at step 1000: 1.843712
Minibatch accuracy: 77.3%
Validation accuracy: 78.3%
Minibatch loss at step 1500: 0.929079
Minibatch accuracy: 83.6%
Validation accuracy: 80.0%
Minibatch loss at step 2000: 0.822873
Minibatch accuracy: 85.9%
Validation accuracy: 80.8%
Minibatch loss at step 2500: 0.865466
Minibatch accuracy: 80.5%
Validation accuracy: 81.0%
Minibatch loss at step 3000: 0.779086
Minibatch accuracy: 84.4%
Validation accuracy: 81.6%
Test accuracy: 88.7%


In [6]:
# 1 hidden layer
# define the graph
hidden_nodes = 1024
batch_size = 128
lamb = 0.001

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, 
                                     shape = (batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights1 = tf.Variable(tf.truncated_normal([image_size*image_size, hidden_nodes]))
    bias1 = tf.Variable(tf.zeros([hidden_nodes]))
    
    weights2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    bias2 = tf.Variable(tf.zeros([num_labels]))
    
    def get_output_with_hidden_levels(input_set, hidden_nodes = hidden_nodes):
        hidden_output = tf.nn.relu(tf.matmul(input_set, weights1) + bias1)
        logits = tf.matmul(hidden_output, weights2) + bias2
        
        return logits
    
    logits = get_output_with_hidden_levels(tf_train_dataset)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels, logits = logits)
      + lamb * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2)))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(get_output_with_hidden_levels(tf_valid_dataset))
    test_prediction = tf.nn.softmax(get_output_with_hidden_levels(tf_test_dataset))
    
# now run it
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Inited')
    for step in xrange(num_steps):
        offset = (step*batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset+batch_size), :]
        batch_labels = train_labels[offset:(offset+batch_size), :]
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Inited
Minibatch loss at step 0: 670.971069
Minibatch accuracy: 13.3%
Validation accuracy: 27.0%
Minibatch loss at step 500: 197.680481
Minibatch accuracy: 80.5%
Validation accuracy: 77.7%
Minibatch loss at step 1000: 116.034271
Minibatch accuracy: 78.1%
Validation accuracy: 81.6%
Minibatch loss at step 1500: 68.674294
Minibatch accuracy: 89.1%
Validation accuracy: 82.9%
Minibatch loss at step 2000: 41.117863
Minibatch accuracy: 91.4%
Validation accuracy: 85.0%
Minibatch loss at step 2500: 25.125443
Minibatch accuracy: 89.1%
Validation accuracy: 85.7%
Minibatch loss at step 3000: 15.439608
Minibatch accuracy: 88.3%
Validation accuracy: 86.8%
Test accuracy: 93.3%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [7]:
# 1 hidden layer
# define the graph
hidden_nodes = 1024
batch_size = 128
lamb = 0.001

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, 
                                     shape = (batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights1 = tf.Variable(tf.truncated_normal([image_size*image_size, hidden_nodes]))
    bias1 = tf.Variable(tf.zeros([hidden_nodes]))
    
    weights2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    bias2 = tf.Variable(tf.zeros([num_labels]))
    
    def get_output_with_hidden_levels(input_set, hidden_nodes = hidden_nodes):
        hidden_output = tf.nn.relu(tf.matmul(input_set, weights1) + bias1)
        logits = tf.matmul(hidden_output, weights2) + bias2
        
        return logits
    
    logits = get_output_with_hidden_levels(tf_train_dataset)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels, logits = logits)
      + lamb * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2)))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(get_output_with_hidden_levels(tf_valid_dataset))
    test_prediction = tf.nn.softmax(get_output_with_hidden_levels(tf_test_dataset))
    
# now run it
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Inited')
    for step in xrange(num_steps):
        offset = (step*batch_size) % (50 * batch_size) # only first 50 batches
        batch_data = train_dataset[offset:(offset+batch_size), :]
        batch_labels = train_labels[offset:(offset+batch_size), :]
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Inited
Minibatch loss at step 0: 708.438721
Minibatch accuracy: 10.9%
Validation accuracy: 43.2%
Minibatch loss at step 500: 189.543365
Minibatch accuracy: 98.4%
Validation accuracy: 79.7%
Minibatch loss at step 1000: 114.846329
Minibatch accuracy: 100.0%
Validation accuracy: 80.2%
Minibatch loss at step 1500: 69.638474
Minibatch accuracy: 100.0%
Validation accuracy: 80.2%
Minibatch loss at step 2000: 42.226852
Minibatch accuracy: 100.0%
Validation accuracy: 80.5%
Minibatch loss at step 2500: 25.616661
Minibatch accuracy: 100.0%
Validation accuracy: 81.1%
Minibatch loss at step 3000: 15.562052
Minibatch accuracy: 100.0%
Validation accuracy: 81.8%
Test accuracy: 89.2%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [8]:
# 1 hidden layer
# define the graph
hidden_nodes = 1024
batch_size = 128
lamb = 0.00

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, 
                                     shape = (batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights1 = tf.Variable(tf.truncated_normal([image_size*image_size, hidden_nodes]))
    bias1 = tf.Variable(tf.zeros([hidden_nodes]))
    
    weights2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    bias2 = tf.Variable(tf.zeros([num_labels]))
    
    def get_output_with_hidden_levels(input_set, hidden_nodes = hidden_nodes, drop = False):
        hidden_output = tf.nn.relu(tf.matmul(input_set, weights1) + bias1)
        if drop:
            hidden_output = tf.nn.dropout(hidden_output, keep_prob = 0.5)
        logits = tf.matmul(hidden_output, weights2) + bias2
        
        return logits
    
    logits = get_output_with_hidden_levels(tf_train_dataset, drop = True)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels, logits = logits)
      + lamb * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2)))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(get_output_with_hidden_levels(tf_train_dataset))
    valid_prediction = tf.nn.softmax(get_output_with_hidden_levels(tf_valid_dataset))
    test_prediction = tf.nn.softmax(get_output_with_hidden_levels(tf_test_dataset))
    
# now run it
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Inited')
    for step in xrange(num_steps):
        offset = (step*batch_size) % (50 * batch_size)#(train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset+batch_size), :]
        batch_labels = train_labels[offset:(offset+batch_size), :]
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Inited
Minibatch loss at step 0: 463.399292
Minibatch accuracy: 10.9%
Validation accuracy: 25.1%
Minibatch loss at step 500: 29.307587
Minibatch accuracy: 92.2%
Validation accuracy: 81.1%
Minibatch loss at step 1000: 3.596560
Minibatch accuracy: 97.7%
Validation accuracy: 82.1%
Minibatch loss at step 1500: 9.967327
Minibatch accuracy: 97.7%
Validation accuracy: 82.2%
Minibatch loss at step 2000: 3.629484
Minibatch accuracy: 99.2%
Validation accuracy: 82.4%
Minibatch loss at step 2500: 0.754715
Minibatch accuracy: 100.0%
Validation accuracy: 82.4%
Minibatch loss at step 3000: 1.368217
Minibatch accuracy: 99.2%
Validation accuracy: 82.5%
Test accuracy: 89.4%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [19]:
# same parameters, try 2 hidden layers
hidden_nodes1 = 1024
hidden_nodes2 = 300
hidden_nodes3 = 50
batch_size = 128
keep_prob = 0.95

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, 
                                     shape = (batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights1 = tf.Variable(tf.truncated_normal([image_size*image_size, hidden_nodes1], stddev=0.1))
    bias1 = tf.Variable(tf.zeros([hidden_nodes1]))
    
    weights2 = tf.Variable(tf.truncated_normal([hidden_nodes1, hidden_nodes2], stddev=0.1))
    bias2 = tf.Variable(tf.zeros([hidden_nodes2]))
    
    weights3 = tf.Variable(tf.truncated_normal([hidden_nodes2, hidden_nodes3], stddev=0.1))
    bias3 = tf.Variable(tf.zeros([hidden_nodes3]))
    
    weights4 = tf.Variable(tf.truncated_normal([hidden_nodes3, num_labels], stddev=0.1))
    bias4 = tf.Variable(tf.zeros([num_labels]))
    
    epsilon = tf.constant(value=0.0001,shape=[num_labels])
    
    def get_output_with_hidden_levels(input_set, drop = False):
        hidden_output1 = tf.nn.relu(tf.matmul(input_set, weights1)) + bias1
        if drop:
            hidden_output1 = tf.nn.dropout(hidden_output1, keep_prob=0.5)
        hidden_output2 = tf.nn.relu(tf.matmul(hidden_output1, weights2)) + bias2
        if drop:
            hidden_output2 = tf.nn.dropout(hidden_output2, keep_prob=0.7)
        hidden_output3 = tf.nn.relu(tf.matmul(hidden_output2, weights3)) + bias3
        if drop:
            hidden_output3 = tf.nn.dropout(hidden_output3, keep_prob=0.9)
        logits = tf.matmul(hidden_output3, weights4) + bias4
        
        return logits
    
    logits = get_output_with_hidden_levels(tf_train_dataset, drop = True)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels, logits = logits))
    
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.001, global_step, 1000, 1)
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss, 
                                                            global_step=global_step)
    
    train_prediction = tf.nn.softmax(get_output_with_hidden_levels(tf_train_dataset))
    valid_prediction = tf.nn.softmax(get_output_with_hidden_levels(tf_valid_dataset))
    test_prediction = tf.nn.softmax(get_output_with_hidden_levels(tf_test_dataset))
    
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Inited')
    for step in xrange(num_steps):
        offset = (step*batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset+batch_size), :]
        batch_labels = train_labels[offset:(offset+batch_size), :]
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Inited
Minibatch loss at step 0: 3.978385
Minibatch accuracy: 10.2%
Validation accuracy: 12.4%
Minibatch loss at step 500: 0.599034
Minibatch accuracy: 84.4%
Validation accuracy: 81.9%
Minibatch loss at step 1000: 0.715771
Minibatch accuracy: 79.7%
Validation accuracy: 83.3%
Minibatch loss at step 1500: 0.471852
Minibatch accuracy: 85.2%
Validation accuracy: 83.9%
Minibatch loss at step 2000: 0.474113
Minibatch accuracy: 92.2%
Validation accuracy: 84.4%
Minibatch loss at step 2500: 0.526393
Minibatch accuracy: 84.4%
Validation accuracy: 85.1%
Minibatch loss at step 3000: 0.618001
Minibatch accuracy: 83.6%
Validation accuracy: 85.2%
Test accuracy: 91.9%
